In [9]:
from IPython.display import Markdown, display


def print_md(string):
    display(Markdown(string))

In [1]:
# 基础代码引入
from dotenv import load_dotenv
import os

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
github_api_key = os.getenv('GITHUB_API_KEY')
github_api_base = os.getenv('GITHUB_API_BASE')

In [2]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    openai_api_base=github_api_base,
    openai_api_key=github_api_key,
    model_name="gpt-4o",
    temperature=0.5,
    streaming=True,
)

In [3]:
llm_openai.invoke("你是谁呀？")

AIMessage(content='我是一个AI助手，专门用来回答问题和提供信息。你有什么需要帮助的吗？', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d54531d9eb'}, id='run-0fc26607-f1b1-466d-88d4-1c8f901a524d-0')

In [4]:
system_template = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()

In [5]:
human_template = """
Task, Goal, or Current Prompt:\n
{question}
"""

In [6]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template),
]
prompt = ChatPromptTemplate.from_messages(messages)

In [7]:
chain = prompt | llm_openai

In [10]:
prompt1 = chain.invoke("Optimize the prompt that users enter for image generation with Flux 1.1 model")

In [16]:
print_md(prompt1.content)

Optimize user prompts for image generation using the Flux 1.1 model by enhancing clarity and specificity to achieve desired results.

# Steps

1. **Understand User Intent**: Identify the main subject, style, and mood the user wants in the image.
2. **Add Specific Details**: Include details such as colors, lighting, background elements, and composition.
3. **Refine Language**: Use precise and descriptive language to convey the desired outcome.
4. **Incorporate Keywords**: Utilize relevant keywords that align with the Flux 1.1 model's capabilities.
5. **Review and Adjust**: Ensure the prompt is clear, concise, and free of ambiguity.

# Output Format

- Provide a refined prompt as a single, coherent sentence.
- Ensure clarity and specificity in the language used.

# Examples

**Example 1:**

- **User Input**: "A sunset over a mountain."
- **Optimized Prompt**: "A vibrant sunset casting warm orange and pink hues over a snow-capped mountain range, with a clear sky and soft shadows."

**Example 2:**

- **User Input**: "A futuristic city."
- **Optimized Prompt**: "A sprawling futuristic cityscape at night, illuminated by glowing neon lights, with towering skyscrapers and flying vehicles."

# Notes

- Focus on enhancing the visual elements described in the prompt.
- Consider the limitations and strengths of the Flux 1.1 model when optimizing prompts.

In [17]:
human_template = """
{question}
"""
messages = [
    SystemMessagePromptTemplate.from_template(prompt1.content),
    HumanMessagePromptTemplate.from_template(human_template),
]
prompt = ChatPromptTemplate.from_messages(messages)

In [18]:
chain = prompt | llm_openai

In [19]:
prompt2 = chain.invoke(
    'a lady with fashion dress, sit by a car. she has a cigaret in her fingers. She wears sunglasses. She is hot')

In [21]:
print_md(prompt2.content)

"A stylish woman in a chic fashion dress sits confidently by a sleek car, holding a cigarette between her fingers, wearing trendy sunglasses, exuding a glamorous and alluring aura."